<a href="https://colab.research.google.com/github/ScriptSherpa/TrackCount360-YOLO-Based-Object-Tracking-Directional-Counting-System/blob/main/YOLO11_Class_Wise_Vehicled_detection_tracking_and_counting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics
import cv2
from ultralytics import YOLO
from collections import defaultdict

# Load the YOLO model
model = YOLO('yolo11n.pt')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.0/973.0 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

100%|██████████| 5.35M/5.35M [00:00<00:00, 168MB/s]


In [ ]:
class_list = model.names
class_list

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [ ]:
# Open the video file
cap = cv2.VideoCapture('/content/test.mp4')

In [ ]:
import cv2
import pandas as pd
from ultralytics import YOLO
from collections import defaultdict

# Load the YOLO model
model = YOLO('yolo11n.pt')

class_list = model.names  # List of class names

# Open the video file
cap = cv2.VideoCapture('test_videos/3.mp4')


# Define line positions for counting
line_y_red = 298  # Red line position
line_y_blue = line_y_red + 100  # Blue line position


# Variables to store counting and tracking information
counted_ids_red_to_blue = set()
counted_ids_blue_to_red = set()

# Dictionaries to count objects by class for each direction
count_red_to_blue = defaultdict(int)  # Moving downwards
count_blue_to_red = defaultdict(int)  # Moving upwards

# State dictionaries to track which line was crossed first
crossed_red_first = {}
crossed_blue_first = {}



# Loop through video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO tracking on the frame
    results = model.track(frame, persist=True)

    # Ensure results are not empty
    if results[0].boxes.data is not None:
        # Get the detected boxes, their class indices, and track IDs
        boxes = results[0].boxes.xyxy.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu()


        # Draw the lines on the frame
        cv2.line(frame, (190, line_y_red), (850, line_y_red), (0, 0, 255), 3)
        cv2.putText(frame, 'Red Line', (190, line_y_red - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

        cv2.line(frame, (27, line_y_blue), (960, line_y_blue), (255, 0, 0), 3)
        cv2.putText(frame, 'Blue Line', (27, line_y_blue - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)


        # Loop through each detected object
        for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
            x1, y1, x2, y2 = map(int, box)

            cx = (x1 + x2) // 2  # Calculate the center point
            cy = (y1 + y2) // 2

            # Get the class name using the class index
            class_name = class_list[class_idx]

            # Draw a dot at the center and display the tracking ID and class name
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)

            cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)


            # Check if the object crosses the red line
            if line_y_red - 5 <= cy <= line_y_red + 5:
                # Record that the object crossed the red line
                if track_id not in crossed_red_first:
                    crossed_red_first[track_id] = True

            # Check if the object crosses the blue line
            if line_y_blue - 5 <= cy <= line_y_blue + 5:
                # Record that the object crossed the blue line
                if track_id not in crossed_blue_first:
                    crossed_blue_first[track_id] = True



            # Counting logic for downward direction (red -> blue)
            if track_id in crossed_red_first and track_id not in counted_ids_red_to_blue:
                if line_y_blue - 5 <= cy <= line_y_blue + 5:
                    counted_ids_red_to_blue.add(track_id)
                    count_red_to_blue[class_name] += 1

            # Counting logic for upward direction (blue -> red)
            if track_id in crossed_blue_first and track_id not in counted_ids_blue_to_red:
                if line_y_red - 5 <= cy <= line_y_red + 5:
                    counted_ids_blue_to_red.add(track_id)
                    count_blue_to_red[class_name] += 1

    # Display the counts on the frame
    y_offset = 30
    for class_name, count in count_red_to_blue.items():
        cv2.putText(frame, f'{class_name} (Down): {count}', (10, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
        y_offset += 30

    y_offset += 20  # Add spacing for upward counts
    for class_name, count in count_blue_to_red.items():
        cv2.putText(frame, f'{class_name} (Up): {count}', (10, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
        y_offset += 30




    # Show the frame
    cv2.imshow("YOLO Object Tracking & Counting", frame)

    # Exit loop if 'ESC' key is pressed
    if cv2.waitKey(0) & 0xFF == 27:
        break

# Release resources
cap.release()

In [ ]:
# Define the video writer to save output
output_path = '/content/output_detected.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Rewind the video for second pass (if needed)
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Loop through video frames again
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO tracking on the frame
    results = model.track(frame, persist=True)

    # Process results exactly like before...
    # [Add your existing detection, line crossing, and counting logic here]

    # Write the processed frame to the output video
    out.write(frame)

# Release everything
cap.release()
out.release()
cv2.destroyAllWindows()

print("✅ Detection and counting done. Output video saved to:", output_path)


✅ Detection and counting done. Output video saved to: /content/output_detected.mp4


In [ ]:
from google.colab import files
files.download('/content/output_detected.mp4')


FileNotFoundError: Cannot find file: /content/output_detected.mp4

In [ ]:
# Define the video writer to save output
output_path = '/content/output_detected.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Rewind the video for second pass (if needed)
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Loop through video frames again
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO tracking on the frame
    results = model.track(frame, persist=True)

    # Process results exactly like before...
    # Ensure results are not empty
    if results[0].boxes.data is not None:
        # Get the detected boxes, their class indices, and track IDs
        boxes = results[0].boxes.xyxy.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu()


        # Draw the lines on the frame
        cv2.line(frame, (190, line_y_red), (850, line_y_red), (0, 0, 255), 3)
        cv2.putText(frame, 'Red Line', (190, line_y_red - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

        cv2.line(frame, (27, line_y_blue), (960, line_y_blue), (255, 0, 0), 3)
        cv2.putText(frame, 'Blue Line', (27, line_y_blue - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)


        # Loop through each detected object
        for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
            x1, y1, x2, y2 = map(int, box)

            cx = (x1 + x2) // 2  # Calculate the center point
            cy = (y1 + y2) // 2

            # Get the class name using the class index
            class_name = class_list[class_idx]

            # Draw a dot at the center and display the tracking ID and class name
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)

            cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)


            # Check if the object crosses the red line
            if line_y_red - 5 <= cy <= line_y_red + 5:
                # Record that the object crossed the red line
                if track_id not in crossed_red_first:
                    crossed_red_first[track_id] = True

            # Check if the object crosses the blue line
            if line_y_blue - 5 <= cy <= line_y_blue + 5:
                # Record that the object crossed the blue line
                if track_id not in crossed_blue_first:
                    crossed_blue_first[track_id] = True



            # Counting logic for downward direction (red -> blue)
            if track_id in crossed_red_first and track_id not in counted_ids_red_to_blue:
                if line_y_blue - 5 <= cy <= line_y_blue + 5:
                    counted_ids_red_to_blue.add(track_id)
                    count_red_to_blue[class_name] += 1

            # Counting logic for upward direction (blue -> red)
            if track_id in crossed_blue_first and track_id not in counted_ids_blue_to_red:
                if line_y_red - 5 <= cy <= line_y_red + 5:
                    counted_ids_blue_to_red.add(track_id)
                    count_blue_to_red[class_name] += 1

    # Display the counts on the frame
    y_offset = 30
    for class_name, count in count_red_to_blue.items():
        cv2.putText(frame, f'{class_name} (Down): {count}', (10, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
        y_offset += 30

    y_offset += 20  # Add spacing for upward counts
    for class_name, count in count_blue_to_red.items():
        cv2.putText(frame, f'{class_name} (Up): {count}', (10, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
        y_offset += 30

    # Write the processed frame to the output video
    out.write(frame)

# Release everything
cap.release()
out.release()
cv2.destroyAllWindows()

print("✅ Detection and counting done. Output video saved to:", output_path)

✅ Detection and counting done. Output video saved to: /content/output_detected.mp4


In [3]:
# Step 1: Install required libraries
!pip install ultralytics opencv-python-headless --quiet

# Step 2: Upload your video
from google.colab import files
uploaded = files.upload()  # Upload your video file  (e.g., 'video.mp4')

# Step 3: YOLOv8 Direction-Based Object Tracking and Counting

import cv2
import pandas as pd
from ultralytics import YOLO
from collections import defaultdict


# Load YOLO model
model = YOLO('yolov8n.pt')  # You can also use 'yolov8s.pt' or your own model
class_list = model.names

# Use uploaded file name
video_path = list(uploaded.keys())[0]
cap = cv2.VideoCapture(video_path)

# Line positions
line_y_red = 300
line_y_blue = line_y_red + 100

# Counting and tracking
counted_ids_red_to_blue = set()
counted_ids_blue_to_red = set()
count_red_to_blue = defaultdict(int)
count_blue_to_red = defaultdict(int)
crossed_red_first = {}
crossed_blue_first = {}

# Output video writer
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter('output_result.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(frame, persist=True, verbose=False)

    if results and results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu().numpy()
        track_ids = results[0].boxes.id.cpu().numpy().astype(int)
        class_indices = results[0].boxes.cls.cpu().numpy().astype(int)

        # Draw lines
        cv2.line(frame, (0, line_y_red), (frame_width, line_y_red), (0, 0, 255), 2)
        cv2.line(frame, (0, line_y_blue), (frame_width, line_y_blue), (255, 0, 0), 2)

        for box, track_id, class_idx in zip(boxes, track_ids, class_indices):
            x1, y1, x2, y2 = map(int, box)
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            class_name = class_list[class_idx]

            # Draw bounding box and info
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, f'ID:{track_id} {class_name}', (x1, y1 - 7),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

            # Track direction
            if line_y_red - 5 <= cy <= line_y_red + 5:
                crossed_red_first[track_id] = True
            if line_y_blue - 5 <= cy <= line_y_blue + 5:
                crossed_blue_first[track_id] = True

            if track_id in crossed_red_first and track_id not in counted_ids_red_to_blue:
                if line_y_blue - 5 <= cy <= line_y_blue + 5:
                    counted_ids_red_to_blue.add(track_id)
                    count_red_to_blue[class_name] += 1

            if track_id in crossed_blue_first and track_id not in counted_ids_blue_to_red:
                if line_y_red - 5 <= cy <= line_y_red + 5:
                    counted_ids_blue_to_red.add(track_id)
                    count_blue_to_red[class_name] += 1

        # Show counts
        y_offset = 30
        for cls, cnt in count_red_to_blue.items():
            cv2.putText(frame, f'{cls} Down: {cnt}', (10, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            y_offset += 25
        for cls, cnt in count_blue_to_red.items():
            cv2.putText(frame, f'{cls} Up: {cnt}', (10, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            y_offset += 25

    out.write(frame)

cap.release()
out.release()

# Step 4: Download result video
from google.colab import files
files.download('output_result.mp4')


Saving people.mp4 to people.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>